In [22]:
# import sys
# !{sys.executable} -m pip install fpl 

In [25]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
from sklearn.preprocessing import MinMaxScaler
import requests
from datetime import datetime as dt
import asyncio
from operator import attrgetter

import aiohttp
from prettytable import PrettyTable

from fpl import FPL
from fpl.utils import team_converter
from colorama import Fore, init

In [26]:
session = requests.session()
url = 'https://users.premierleague.com/accounts/login/'
creds = ("knowingvivek@gmail.com", "Pl1nc#$$")
payload = {
    'password': creds[0],
    'login': creds[1],
    'redirect_uri': 'https://fantasy.premierleague.com/a/login',
    'app': 'plfpl-web'
}
session.post(url, data=payload)
global team_map

In [28]:
def get_json(file_path):
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    jsonResponse = r.json()
    with open(file_path, 'w') as outfile:
        json.dump(jsonResponse, outfile)
get_json('fpl.json')
with open('fpl.json') as json_data:
    d = json.load(json_data)
global team_map
team_map ={}
for rec in d["teams"]:
    name = rec["name"]
    i = rec["id"]
    team_map[i]=name
global pos_map
pos_map = {1: "Goalkeeper", 2: "Defender", 3:"Midfielder", 4:"Forward"}

# fpl library

In [32]:
async with aiohttp.ClientSession() as session:
    fpl = FPL(session)
    f_players = await fpl.get_players()

    player_table = PrettyTable()
    player_table.field_names = ["Player", "£", "Form", "Games" ,"Mins", "G", "A","Clean Sheets",
                                "Points", "VAPM", "Tr_In","Tr_Out"]
    player_table.align["Player"] = "l"

    for player in f_players:
        
        if player.web_name == "Coufal":
            
            break

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [31]:
p.chance_of_playing_next_round

NameError: name 'p' is not defined

In [59]:
p.selected_by_percent

NameError: name 'p' is not defined

In [11]:
def get_gameweek_score(player, gameweek):
    try:
        gameweek_history = [history for history in player.history if history["round"]==gameweek][0]
    except:
        return 0
    return gameweek_history["total_points"]

def get_gameweek_opponent(player, gameweek):
    gameweek_history = next(history for history in player.history
                            if history["round"] == gameweek)
    return (f"{team_converter(gameweek_history['opponent_team'])} ("
            f"{'H' if gameweek_history['was_home'] else 'A'})")

def get_point_difference(player_a, player_b, gameweek):
    if player_a == player_b:
        return 0

    history_a = next(history for history in player_a.history
                    if history["round"] == gameweek)
    history_b = next(history for history in player_b.history
                    if history["round"] == gameweek)

    return history_a["total_points"] - history_b["total_points"]

async def captain_performance(user_id=285714):
    player_table = PrettyTable()
    player_table.field_names = ["Gameweek", "Captain", "Top scorer", "Δ"]
    player_table.align = "r"
    total_difference = 0

    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        user = await fpl.get_user(user_id)
        picks = await user.get_picks()
        for i, elements in enumerate(picks):
            elements = picks[elements]
            gameweek = i + 1
            captain_id = next(player for player in elements
                              if player["is_captain"])["element"]
            players = await fpl.get_players(
                [player["element"] for player in elements],
                include_summary=True)

            captain = next(player for player in players
                          if player.id == captain_id)

            top_scorer = max(
                players, key=lambda x: get_gameweek_score(x, gameweek))

            point_difference = get_point_difference(
                captain, top_scorer, gameweek)

            player_table.add_row([
                gameweek,
                (f"{captain.web_name} - "
                f"{get_gameweek_score(captain, gameweek)} points vs. "
                f"{get_gameweek_opponent(captain, gameweek)}"),
                (f"{top_scorer.web_name} - "
                f"{get_gameweek_score(top_scorer, gameweek)} points vs. "
                f"{get_gameweek_opponent(top_scorer, gameweek)}"),
                point_difference
            ])

            total_difference += point_difference

    print(player_table)
    print(f"Total point difference is {abs(total_difference)} points!")
    
async def top_performer(n = 10):
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        players = await fpl.get_players()

    top_performers = sorted(
        players, key=lambda x: x.goals_scored + x.assists, reverse=True)

    player_table = PrettyTable()
    player_table.field_names = ["Player", "£", "G", "A", "G + A"]
    player_table.align["Player"] = "l"

    for player in top_performers[:n]:
        goals = player.goals_scored
        assists = player.assists
        player_table.add_row([player.web_name, f"£{player.now_cost / 10}",
                            goals, assists, goals + assists])

    print(player_table)
    
async def player_performance(players = []):
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        f_players = await fpl.get_players()

        player_table = PrettyTable()
        player_table.field_names = ["Player", "£", "Form", "Games" ,"Mins", "G", "A","Clean Sheets",
                                    "Points", "VAPM", "%Selected","Fitness"]
        player_table.align["Player"] = "l"

        for player in f_players:
            if player.web_name in players:
                goals = player.goals_scored
                assists = player.assists
                if player.chance_of_playing_next_round is None:
                    player.chance_of_playing_next_round = 100
                player_table.add_row([player.web_name, f"£{player.now_cost / 10}",
                                    player.form,await player.games_played,player.minutes,goals,
                                      assists, player.clean_sheets,player.total_points,round(await player.vapm,2)
                                      ,player.selected_by_percent,player.chance_of_playing_next_round])

        print(player_table)

async def fdr_ptable():
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        fdr = await fpl.FDR()

    fdr_table = PrettyTable()
    fdr_table.field_names = [
        "Team", "All (H)", "All (A)", "GK (H)", "GK (A)", "DEF (H)", "DEF (A)",
        "MID (H)", "MID (A)", "FWD (H)", "FWD (A)"]

    for team, positions in fdr.items():
        row = [team]
        for difficulties in positions.values():
            for location in ["H", "A"]:
                if difficulties[location] == 5.0:
                    row.append(Fore.RED + "5.0" + Fore.RESET)
                elif difficulties[location] == 1.0:
                    row.append(Fore.GREEN + "1.0" + Fore.RESET)
                else:
                    row.append(f"{difficulties[location]:.2f}")

        fdr_table.add_row(row)

    fdr_table.align["Team"] = "l"
    print(fdr_table)

    async def get_fdr_diff(team: None):
        global team_map
        async with aiohttp.ClientSession() as session:
            fpl = FPL(session)
            fdr = await fpl.FDR()
            t_detail = await fpl.get_team(team)
            t_is_home = (await t_detail.get_fixtures())[0]["is_home"]
            if t_is_home:
                team2 = (await t_detail.get_fixtures())[0]["team_a"]
                team=team_map[team]
                team2=team_map[team2]
                team = fdr[team]["all"]["H"]
                team2 = fdr[team2]["all"]["A"]
            else:
                team2 = (await t_detail.get_fixtures())[0]["team_h"]
                team=team_map[team]
                team2=team_map[team2]
                team = fdr[team]["all"]["A"]
                team2 = fdr[team2]["all"]["H"]
            return round(team2-team,2)
        
async def next_fixture_difficulty(teams = None,gws=None, zone="all", scaled=False):
    
    global team_map
    ndf = pd.DataFrame(index = teams)
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        fdr = await fpl.FDR()
        for team in team_map.keys():
            team_name=team_map[team]
            if teams is not None and team_name not in teams:
                continue
            t_detail = await fpl.get_team(team)
            fixtures = await t_detail.get_fixtures()
            
            
            if gws is not None:
                fixtures = fixtures[:gws]

            for fix in fixtures:
                try:
                    gameweek = int(fix["event_name"][-2:])
                    t_is_home = fix["is_home"]
                    if t_is_home:
                        team2 = fix["team_a"]
                        team2_name=team_map[team2]
                        
                        if zone == "defence":
                            team_fdr = (fdr[team_name]["goalkeeper"]["H"]+fdr[team_name]["defender"]["H"])/2
                            team2_fdr = (fdr[team2_name]["midfielder"]["A"]+fdr[team2_name]["forward"]["A"])/2
                        elif zone == "attack":
                            team_fdr = (fdr[team_name]["midfielder"]["H"]+fdr[team_name]["forward"]["H"])/2
                            team2_fdr = (fdr[team2_name]["goalkeeper"]["A"]+fdr[team2_name]["defender"]["A"])/2
                        else:
                            team_fdr = fdr[team_name]["all"]["H"]
                            team2_fdr = fdr[team2_name]["all"]["A"]
                    else:
                        team2 = fix["team_h"]
                        team2_name=team_map[team2]
                        if zone == "defence":
                            team_fdr = (fdr[team_name]["goalkeeper"]["A"]+fdr[team_name]["defender"]["A"])/2
                            team2_fdr = (fdr[team2_name]["midfielder"]["H"]+fdr[team2_name]["forward"]["H"])/2
                        elif zone == "attack":
                            team_fdr = (fdr[team_name]["midfielder"]["A"]+fdr[team_name]["forward"]["A"])/2
                            team2_fdr = (fdr[team2_name]["goalkeeper"]["H"]+fdr[team2_name]["defender"]["H"])/2
                        else:
                            team_fdr = fdr[team_name]["all"]["A"]
                            team2_fdr = fdr[team2_name]["all"]["H"]
                    score = (team_fdr - team2_fdr)
                    try:
                        if ndf.at[team_name, gameweek] >= 0:
                            print(team_name,",",team2_name,",",gameweek,score)
                            #ndf.at[team_name, gameweek] = score
                            pass
                    except:
                        pass
                        
                    ndf.at[team_name,gameweek] = (5-round(score,2))
                except:
                    pass
    if scaled and teams is None:
        ndf = ndf.reindex(sorted(ndf.columns), axis=1)
        for idx, col in enumerate(ndf.columns):
            try:
                values = ndf[col].values
                scaled = MinMaxScaler(feature_range=(0,5)).fit_transform(values.reshape(-1, 1))
                scaled = [round(x[0],2) for x in scaled]
                ndf[col] = scaled
            except:
                print(col)
    col = "Average"
    ndf[col] = ndf.mean(axis=1)
    
    
    return ndf.fillna("-")

async def get_vapm_all_players(top=None, pos=list(pos_map.values())):
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        players = await fpl.get_players()
        team = await fpl.get_teams()
        global team_map
        global pos_map
        pos_codes = [k for k,v in pos_map.items() if v in pos]
    
        top_performers = sorted(
            players, key=lambda x: ((float(x.points_per_game)-2.0)/x.now_cost/10), reverse=True)

        player_table = PrettyTable()
        player_table.field_names = ["Player", "Team", "Pos", "Form", "£", "G", "A", "PPM", "VAPM"]
        player_table.align["Player"] = "l"

        for player in top_performers:
            if player.element_type in pos_codes:
                goals = player.goals_scored
                assists = player.assists
                ppm = round(player.total_points/(player.now_cost),2)
        
                player_table.add_row([player.web_name,team_map[player.team],pos_map[player.element_type],player.form,f"£{player.now_cost/10}",
                            goals, assists, ppm, round(await player.vapm,2)])
        if top is None:
            print(player_table)
        else:
            print(player_table.get_string(start=0, end=top))
            
async def get_best_vapm_players(teams= None, top=None, pos=list(pos_map.values())):
    if teams is None:
        await get_vapm_all_players(top,pos)
        return
    global team_map
    global pos_map
    pos_codes = [k for k,v in pos_map.items() if v in pos]
    team_codes = []
    for team in teams: 
        try:
            team_code = [code for code,name in team_map.items() if name==team][0]
            team_codes.append(team_code)
        except:
            print(team," team not found!")
            return
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        players = await fpl.get_players()
        team = await fpl.get_teams()
    
        top_performers = sorted(players, key=lambda x: ((float(x.points_per_game)-2.0)/(x.now_cost/10)), reverse=True)
        player_table = PrettyTable()
        player_table.field_names = ["Player", "Team", "Pos", "Form", "£", "G", "A", "PPM", "VAPM"]
        player_table.align["Player"] = "l"
        
        i = 0
        for player in top_performers:
            if player.chance_of_playing_this_round is None:
                continue
            if (player.team in team_codes) and (player.chance_of_playing_this_round >=75) and (player.element_type in pos_codes):
                goals = player.goals_scored
                assists = player.assists
                ppm = round(player.total_points/(player.now_cost),2)
        
                player_table.add_row([player.web_name,team_map[player.team],pos_map[player.element_type],player.form,f"£{player.now_cost/10}", goals, assists, ppm, round(await player.vapm,2)])
                i+=1
            if i>=top:
                break
            
        print(player_table)
            
async def my_team(user_id = 285714, summary=False):
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        await fpl.login(*creds)
        user = await fpl.get_user(user_id)
        
        transfers_in = await fpl.get_players(
                [player["element_in"] for player in (await user.get_latest_transfers())],
                include_summary=True)
        
        transfers_out = await fpl.get_players(
            [player["element_out"] for player in (await user.get_latest_transfers())],
            include_summary=True)
        
        
        curr_team = await fpl.get_players(
                [player["element"] for player in (await user.get_team())],
                include_summary=True)
        if not summary:
            print("\nTransfers In:")
            for p in transfers_in:
                print(p.web_name)
            print("\nTransfers Out:")
            for p in transfers_out:
                print(p.web_name)
            print("\nCurrent Squad:")
            for p in curr_team:
                print(p.web_name)
        else:
            fields = ["Player", "Team", "Pos", "Minutes", "Form", "VAPM" ,"Injury"]
            
            n_fdr = await next_fixture_difficulty()
            fields = fields + list(n_fdr.columns)
            summary_df = pd.DataFrame(columns=fields)
            for p in curr_team:
                team = team_map[p.team]
                injury = p.chance_of_playing_next_round
                if injury is None:
                    injury = 100
                summary_df.loc[len(summary_df)]=[p.web_name,team_map[p.team],pos_map[p.element_type],p.minutes,p.form,round(await p.vapm,4),injury, *n_fdr.loc[team].values]
                
            return summary_df
        
async def team_ep_this(user_id = 285714):
    global pos_map
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        await fpl.login(*creds)
        user = await fpl.get_user(user_id)
        
        curr_team = await fpl.get_players(
                [player["element"] for player in (await user.get_team())],
                include_summary=True)

        fields = ["Player", "Team", "VAPM", "Pos","Expected Points", "Injury"]

        summary_df = pd.DataFrame(columns=fields)
        for p in curr_team:
            team = team_map[p.team]
            injury = p.chance_of_playing_next_round
            if injury is None:
                injury = 100
            summary_df.loc[len(summary_df)]=[p.web_name,team_map[p.team],round(await p.vapm,4),
                                             pos_map[p.element_type],float(p.ep_this),injury]
#         d = 0.0
#         t = 0.0
#         for name,g in df.groupby(["Pos"]):
#             d+=min(g["Expected Points"].values)
#             t+=sum(g["Expected Points"].values)
        summary_df.drop(["Pos"],axis=1,inplace=True)
        summary_df.loc[len(summary_df)]=["Total","-","-",sum(summary_df["Expected Points"].values),"-"]
        return summary_df

# Execution

In [14]:
await captain_performance()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [22]:
await fdr_ptable()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [23]:
await next_fixture_difficulty(gws=4,zone="all",scaled=True)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [24]:
await next_fixture_difficulty(teams=["West Ham","Leeds"], gws=4, zone="attack")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [25]:
await get_best_vapm_players(teams=None,top=25,pos=[
#                                                     "Defender",
                                                   "Midfielder",
 #                                                   "Forward"
                                                  ])

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [26]:
await player_performance(["Rashford"])

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [27]:
await get_vapm_all_players(top=20)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [28]:
await my_team(summary=True)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [29]:
await team_ep_this()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [30]:
async with aiohttp.ClientSession() as session:
    fpl = FPL(session)
    f_players = await fpl.get_players()
    for player in f_players:
        if player.web_name == "Pulisic":
            break

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [31]:
player.ep_this,player.ep_next

NameError: name 'player' is not defined